# Выгрузка данных

In [1]:
# Импорт необходимых для работы библиотек
import requests
import sqlite3

In [2]:
# получили Json с данными путем открытия на сайте KFC devtools-Network-store.get_restaurants?showClosed=true
KFC_API_URL = 'https://api.kfc.digital/api/store/v2/store.get_restaurants?showClosed=true'

In [3]:
# Изпользуя метод get() извлечем данные из ресурса
response = requests.get(KFC_API_URL)

In [4]:
# raise_for_status() -метод позволяющий вызвать исключение HTTPError.
response.raise_for_status()

# json() метод возвращающий закодированные в json данные,сохраним их в переменную data
data = response.json()

In [5]:
# data - представляет собой словарь, выведем основные ключи словаря и выберем нудный для дальнейшей работы
data.keys()
rests_data = data['searchResults']

# выведем данные на экран
# print (rests_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Преобработка данных

In [6]:
# Из общего списка данных, выберем нужные параметры и исключим строки, с пропусками необходимых данных

data_to_insert = []
for row_rest_data in rests_data:
    rest_data = row_rest_data['storePublic']
    real_rest_data = {}
    
    if rest_data['title'].get('ru') is None:
        continue
    real_rest_data['title'] = rest_data['title']['ru']
    
    if rest_data['contacts']['city'].get('ru') is None:
        continue
    real_rest_data['city'] = rest_data['contacts']['city']['ru']
    

    if rest_data['contacts']['coordinates']['geometry'].get('coordinates') is None:
        continue
    real_rest_data['coordinates'] = rest_data['contacts']['coordinates']['geometry']['coordinates']
    
    
    if rest_data['contacts']['streetAddress'].get('ru') is None:
        continue
    real_rest_data['address'] = rest_data['contacts']['streetAddress']['ru']
    
    
    if rest_data['openingHours']['regular'].get('startTimeLocal') is None:
        continue
    real_rest_data['openingHours'] = rest_data['openingHours']['regular']['startTimeLocal']
    

    real_rest_data['features'] = rest_data['features']
    
    #Сохраним в пустой список отобранные данные
    data_to_insert.append(real_rest_data)

In [7]:
# Сохраним в переменную отобранные для дальнейшей работы ключи
keys = ['title','city','coordinates', 'address', 'openingHours','features']

In [8]:
# Вызовим новую базу данных, чтобы сохранить в ней данные по ресторанм
con = sqlite3.connect('KFC_rest_info')

# Курсор необходим для выполнения инструкций SQL и извлечения результатов из SQL-запросов 
cur = con.cursor()

In [9]:
# При повторном запуске кода может возниктуть ошибка создания таблицы в SQL, так как они уже будет существовать, применим конструкцию try,except
try:
    cur.execute("CREATE TABLE kfc({})".format(','.join(keys)))

except sqlite3.OperationalError:
    pass
    

In [10]:
# Вызовем con.commit()объекта connection для фиксации транзакции и сохранения информации в базе данных
con.commit()

In [11]:
# Сохраним в переменную многострочную строку
insert_query = """INSERT INTO {table_name} ({columns})
VALUES
"""

In [12]:
# добавим в insert_query данные в формате строк,и вынесем каждую запись на новую строку, чтобы в дальнейшем добавишь данные в БВ
for i, row_rest in enumerate(data_to_insert):
    values = list()
    values.append("'" + str(row_rest['title']) + "'")
    values.append("'" + str(row_rest['city']) + "'")
    values.append("'" + str(row_rest['coordinates']) + "'")
    values.append("'" + str(row_rest['address']) + "'")
    values.append("'" + str(row_rest['openingHours']) + "'")
    values.append("'" + ','.join(row_rest['features'])   + "'")
    
    insert_query += '(' + ','.join(values) + ')'
    if i != len(data_to_insert) - 1:
        insert_query += ',\n'

In [13]:
# сохраним в таблицу полученные в insert_query данные
real_insert_query = insert_query.format(table_name='kfc', columns=','.join(keys))

In [14]:
# выведим таблицу на экран
# real_insert_query

'INSERT INTO kfc (title,city,coordinates,address,openingHours,features)\nVALUES\n(\'KFC Победа Пушкино Москва\',\'Московская область\',\'[56.011733, 37.847461]\',\'141207, Пушкино, Московский проспект, 7а\',\'08:00:00\',\'wifi,breakfast\'),\n(\'KFC Георгиевск Георгиевск\',\'Георгиевск\',\'[44.130827, 43.446866]\',\'357820 г. Георгиевск, ул. Калинина, 150\xa0Б.\',\'09:00:00\',\'driveIn,breakfast\'),\n(\'KFC Скала Самара\',\'Самара\',\'[53.203554, 50.147112]\',\'443013, Самара, Московское шоссе, 4с15\',\'09:00:00\',\'wifi\'),\n(\'KFC Советская Майкоп\',\'Майкоп\',\'[44.605503, 40.106292]\',\'385000, Майкоп, Советская, 201/19\',\'09:00:00\',\'wifi,breakfast\'),\n(\'KFC АЗС Лукойл Химки Москва\',\'Московская область\',\'[55.908962, 37.407612]\',\'141402, Химки, Ленинградское шоссе, 23-й километр, корпус 36\',\'03:00:00\',\'driveIn,wifi,breakfast\'),\n(\'KFC КомсоМолл Иркутск\',\'Иркутск\',\'[52.267986, 104.288963]\',\'664022, Иркутск, Верхняя набережная, 10\',\'10:00:00\',\'breakfast\'),\n

In [15]:
# Перед добавлением данных в базу данных необходимо вызвать курсор
cur = con.cursor()

In [16]:
# Вызовем метод execute чтобы добавить данные в базу данных cur.execute(real_insert_query)

In [17]:
# Чтобы при повторном запуске кода, строки не повторялись запишим if, который будет считать количество строк в таблице
cur.execute("SELECT COUNT(*) from kfc")
table_check = cur.fetchall()

if table_check[0][0] == 0:
    cur.execute(real_insert_query)
    
print(table_check)    

[(1146,)]


In [18]:
cur = con.cursor()

In [19]:
con.commit()

In [20]:
# Выведем на экран получившуюся базу данных по ресторанам KF
for row in cur.execute("SELECT * FROM kfc"):
    print(row)

('KFC Победа Пушкино Москва', 'Московская область', '[56.011733, 37.847461]', '141207, Пушкино, Московский проспект, 7а', '08:00:00', 'wifi,breakfast')
('KFC Георгиевск Георгиевск', 'Георгиевск', '[44.130827, 43.446866]', '357820 г. Георгиевск, ул. Калинина, 150\xa0Б.', '09:00:00', 'driveIn,breakfast')
('KFC Скала Самара', 'Самара', '[53.203554, 50.147112]', '443013, Самара, Московское шоссе, 4с15', '09:00:00', 'wifi')
('KFC Советская Майкоп', 'Майкоп', '[44.605503, 40.106292]', '385000, Майкоп, Советская, 201/19', '09:00:00', 'wifi,breakfast')
('KFC АЗС Лукойл Химки Москва', 'Московская область', '[55.908962, 37.407612]', '141402, Химки, Ленинградское шоссе, 23-й километр, корпус 36', '03:00:00', 'driveIn,wifi,breakfast')
('KFC КомсоМолл Иркутск', 'Иркутск', '[52.267986, 104.288963]', '664022, Иркутск, Верхняя набережная, 10', '10:00:00', 'breakfast')
('KFC Рамстор Сити Москва', 'Москва', '[55.880997, 37.449501]', '125445, Москва, Правобережная, 1', '10:00:00', 'wifi,breakfast')
('KFC

# Создание SQL запроса по параметрам задания 

In [21]:
# Учитывая условия задачи создаим запрос базе данных с необходимыми условиямиц

for row in cur.execute(
    "SELECT * FROM kfc WHERE city = 'Новосибирск' AND openingHours < '08:45:00' AND  features LIKE '%breakfast%'"

):
    print(row)

('KFC Искра Новосибирск', 'Новосибирск', '[54.968897, 82.872774]', '630079, Новосибирск, Станиславского, 40/1', '08:00:00', 'wifi,breakfast')
('KFC Красный Новосибирск', 'Новосибирск', '[55.031273, 82.919411]', '630099, Новосибирск, Красный проспект, 29', '00:00:00', '24hours,wifi,breakfast')
('KFC Фестиваль Новосибирск', 'Новосибирск', '[54.983633, 82.892683]', '630078, Новосибирск, площадь Карла Маркса, 2', '08:00:00', 'wifi,breakfast')
('KFC Высоцкого Новосибирск', 'Новосибирск', '[55.029781, 83.018304]', '630133, Новосибирск, Высоцкого, 45/1', '08:00:00', 'wifi,breakfast')
('KFC Лукойл Военная Новосибирск', 'Новосибирск', '[55.027175, 82.93854]', '630017, Новосибирск, Военная, 7/1', '08:00:00', 'driveIn,wifi,breakfast')
('KFC ТДН Новосибирск', 'Новосибирск', '[55.035621, 82.900522]', '630004, Новосибирск, Вокзальная магистраль, 1', '00:00:00', '24hours,wifi,breakfast')
('KFC Троллейная Новосибирск', 'Новосибирск', '[54.964728, 82.853306]', '630107, Новосибирск, Троллейная, 130', '0